Задание
Цель:
Применить на практике алгоритмы по автоматической оптимизации параметров моделей машинного обучения.
Описание задания:
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная, наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.
Этапы работы:

1. Получите данные и загрузите их в рабочую среду.
2. Подготовьте датасет к обучению моделей:
a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.
3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.
4. Обучите модель логистической регрессии с параметрами по умолчанию.
5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])
6. Оптимизируйте 3-4 параметра модели:
a) Используйте GridSearchCV.
b) Используйте RandomizedSearchCV.
c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.
d) Повторите п. 5 после каждого итогового изменения параметров.
7. Сформулируйте выводы по проделанной работе:
a) Сравните метрики построенных моделей.
b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».
Для получения зачета по этому домашнему заданию минимально необходимо:
обучить одну модель классификации;
оптимизировать параметры, используя метод из п. 6a;
вывести значения метрик.
Результат:
Получены знания по оптимизации параметров.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import uniform 
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Data/heart.csv')

In [3]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [4]:
# категориальные переменные переведем в числовые
X = pd.get_dummies(df, columns=['Sex', 'ChestPainType', 'Oldpeak', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype=int)
# столбец HeartDisease является целевой переменной, удаляем его из Х
del X['HeartDisease']
X.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,...,Oldpeak_5.6,Oldpeak_6.2,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1,0,0,0,1,...,0,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0,1,0,1,0,...,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0,1,0,0,1,...,0,0,0,1,0,1,0,0,0,1


In [5]:
y = df['HeartDisease']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
model = LogisticRegression(random_state=42)

In [8]:
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [44]:
score_clean_model = model.score(X_test, y_test)
score_clean_model

0.8478260869565217

In [10]:
report = classification_report(y_test, model.predict(X_test), target_names=['Non-churned', 'Churned'])
print(report)

              precision    recall  f1-score   support

 Non-churned       0.79      0.87      0.83        77
     Churned       0.90      0.83      0.86       107

    accuracy                           0.85       184
   macro avg       0.84      0.85      0.85       184
weighted avg       0.85      0.85      0.85       184


In [11]:
cross_validate(model, X_test, y_test, cv=10, scoring=['accuracy', 'recall', 'precision', 'f1'])
#так себе наглядность параметров...

{'fit_time': array([0.04058266, 0.03978324, 0.0405519 , 0.04142737, 0.05238962,
        0.04945111, 0.03780532, 0.03843331, 0.04882741, 0.05141544]),
 'score_time': array([0.0085721 , 0.01116085, 0.00833178, 0.00854588, 0.01252103,
        0.00676465, 0.01111698, 0.00163388, 0.        , 0.00861073]),
 'test_accuracy': array([0.89473684, 0.78947368, 0.84210526, 0.94736842, 0.77777778,
        1.        , 0.83333333, 0.72222222, 0.83333333, 0.88888889]),
 'test_recall': array([1.        , 0.90909091, 0.81818182, 1.        , 0.8       ,
        1.        , 0.8       , 0.81818182, 0.81818182, 0.90909091]),
 'test_precision': array([0.84615385, 0.76923077, 0.9       , 0.91666667, 0.8       ,
        1.        , 0.88888889, 0.75      , 0.9       , 0.90909091]),
 'test_f1': array([0.91666667, 0.83333333, 0.85714286, 0.95652174, 0.8       ,
        1.        , 0.84210526, 0.7826087 , 0.85714286, 0.90909091])}

In [12]:
grid = GridSearchCV(model, param_grid = {'penalty': ('l1', 'l2', 'elasticnet', None),'C': [1, 10], 'solver': ('lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga')}, cv=10, scoring='accuracy', n_jobs=-1)

In [13]:
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'C': [1, 10],
                         'penalty': ('l1', 'l2', 'elasticnet', None),
                         'solver': ('lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga')},
             scoring='accuracy')

In [14]:
print(grid.best_score_)
print(grid.best_estimator_)

0.85816364309515
LogisticRegression(C=10, penalty='l1', random_state=42, solver='liblinear')


In [35]:
score_grid = grid.score(X_test, y_test)
score_grid

0.8315217391304348

In [16]:
report = classification_report(y_test, grid.predict(X_test), target_names=['Non-churned', 'Churned'])
print(report)

              precision    recall  f1-score   support

 Non-churned       0.76      0.88      0.81        77
     Churned       0.90      0.79      0.85       107

    accuracy                           0.83       184
   macro avg       0.83      0.84      0.83       184
weighted avg       0.84      0.83      0.83       184


In [17]:
param_grid = {'penalty': ('l1', 'l2', 'elasticnet', None),'C': [1, 10], 'solver': ('lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga')}
grid_random = RandomizedSearchCV(model, param_grid, cv=10, scoring='accuracy')

grid_random.fit(X_train,y_train)

RandomizedSearchCV(cv=10, estimator=LogisticRegression(random_state=42),
                   param_distributions={'C': [1, 10],
                                        'penalty': ('l1', 'l2', 'elasticnet',
                                                    None),
                                        'solver': ('lbfgs', 'liblinear',
                                                   'newton-cg',
                                                   'newton-cholesky', 'sag',
                                                   'saga')},
                   scoring='accuracy')

In [18]:
print(grid_random.best_score_)
print(grid_random.best_estimator_)

0.8567937800814512
LogisticRegression(C=10, random_state=42, solver='newton-cholesky')


In [36]:
score_grid_random = grid_random.score(X_test, y_test)
score_grid_random

0.8315217391304348

In [20]:
report = classification_report(y_test, grid_random.predict(X_test), target_names=['Non-churned', 'Churned'])
print(report)

              precision    recall  f1-score   support

 Non-churned       0.76      0.88      0.81        77
     Churned       0.90      0.79      0.85       107

    accuracy                           0.83       184
   macro avg       0.83      0.84      0.83       184
weighted avg       0.84      0.83      0.83       184


In [21]:
models=[
      {'name':'NB',"model":BernoulliNB(), 'params':{'alpha': uniform(loc=0, scale=4)}},  
      {'name':'Lr',"model": LogisticRegression()  , 'params':{'C':[0.1,0.2,0.3,0.5,0.7,1], 'penalty':['l1', 'l2']}},
      {'name':'SVC',"model": SVC(), 'params':{'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['scale', 'auto']}},
      {'name':'RF',"model": RandomForestClassifier(), 'params':{'n_estimators':[10,25,50,100,150,200], 'criterion':['gini', 'entropy'], 'max_depth':[3,5,7,9,11]}},
      {'name':'DT',"model": DecisionTreeClassifier(), 'params':{'criterion':['gini', 'entropy'], 'max_depth':[3,5,7,9,11]}}]

In [22]:
res=[]
for v in  models:
    res.append((v['name'], RandomizedSearchCV(v['model'], v['params'], cv=10, n_jobs=-1).fit(X_train, y_train)))

In [23]:
for r in res:
    print(r[0], r[1].best_score_, r[1].best_params_)

NB 0.8569233617178822 {'alpha': 3.132510237936054}
Lr 0.8595335061088486 {'penalty': 'l2', 'C': 0.2}
SVC 0.8609033691225472 {'kernel': 'linear', 'gamma': 'scale'}
RF 0.8745464642724915 {'n_estimators': 200, 'max_depth': 9, 'criterion': 'entropy'}
DT 0.858293224731581 {'max_depth': 5, 'criterion': 'gini'}


In [38]:
result_score = []
for r in res:
    a = r[0], r[1].best_score_
    result_score.append(a)

In [55]:
result_score[1][1]

0.8595335061088486

In [56]:
result = pd.DataFrame()
result['Model'] = ['LogisticRegression_clean', 'GridSearchCV', 'RandomizedSearchCV', 'BernoulliNB', 'LogisticRegression', 'SVC', 'RandomForestClassifier', 'DecisionTreeClassifier']
result['Score'] = [score_clean_model, score_grid, score_grid_random, result_score[0][1], result_score[1][1], result_score[2][1], result_score[3][1], result_score[4][1]]
result

,Model,Score
0,LogisticRegression_clean,0.847826
1,GridSearchCV,0.831522
2,RandomizedSearchCV,0.831522
3,BernoulliNB,0.856923
4,LogisticRegression,0.859534
5,SVC,0.860903
6,RandomForestClassifier,0.874546
7,DecisionTreeClassifier,0.858293


Подбор параметров позволяет улучшить качество модели, но иногда занимает много времени и ресурсов, так подбор для 5 моделей занимал несколько часов, но подбор параметров позволил улучшить качество модели, так, например, в дз "Ансамблирование" RandomForestClassifier имел score = 0.864130, сейчас же, после подбора параметров метрика улучшилась до 0.874546.